In [1]:
from pathlib import Path
from topagnps2cche1d import Watershed, CrossSection, Reach, Node, Cell

Initialize Watershed

In [2]:
watershed = Watershed()

Create Reaches and add to watershed

In [3]:
# Reaches
reach4 = Reach(id=4, receiving_reach_id=2, slope=0.001)

reach3 = Reach(id=3, receiving_reach_id=2, slope=0.001)

reach2 = Reach(id=2, slope=0.001)

Create Nodes and assign them to reaches

In [4]:
# Nodes
n1 = Node(id=1, dsid=2, x=700, y=100)

n2 = Node(id=2, usid=1, dsid=3, x=600, y=200)

n3 = Node(id=3, usid=2, x=500, y=300)

for node in [n1, n2, n3]:
    reach4.add_node(node)

reach4.us_nd_id = 1
reach4.ds_nd_id = 3

n4 = Node(id=4, dsid=5, x=700, y=700)

n5 = Node(id=5, usid=4, dsid=6, x=600, y=600)

n6 = Node(id=6, usid=5, x=500, y=500)

for node in [n4, n5, n6]:
    reach3.add_node(node)

reach3.us_nd_id = 4
reach3.ds_nd_id = 6

n7 = Node(id=7, dsid=8, x=400, y=400)

n8 = Node(id=8, usid=7, dsid=9, x=300, y=400)

n9 = Node(id=9, usid=8, dsid=10, x=200, y=400)

n10 = Node(id=10, usid=9, x=100, y=400)

for node in [n7, n8, n9, n10]:
    reach2.add_node(node)

reach2.us_nd_id = 7
reach2.ds_nd_id = 10

Create cells

In [5]:
c41 = Cell(id=41, receiving_reach_id=4, area=4.1)
c42 = Cell(id=42, receiving_reach_id=4, area=4.2)
c43 = Cell(id=43, receiving_reach_id=4, area=4.3)

c31 = Cell(id=31, receiving_reach_id=3, area=3.1)
c32 = Cell(id=32, receiving_reach_id=3, area=3.2)
c33 = Cell(id=33, receiving_reach_id=3, area=3.3)

c22 = Cell(id=22, receiving_reach_id=2, area=2.2)
c23 = Cell(id=23, receiving_reach_id=2, area=2.3)

Assign reaches and cells to watershed

In [6]:
for reach in [reach2, reach3, reach4]:
    watershed.add_reach(reach)

for cell in [c41, c42, c43, c31, c32, c33, c22, c23]:
    watershed.add_cell(cell)

Finish setting up watershed

In [7]:
watershed.update_graph()
watershed.determine_reaches_us_ds_direction()
watershed.assign_strahler_number_to_reaches()
watershed.ignore_reaches_with_strahler_leq(1)

# It's important to ignore reaches BEFORE proper junctions are defined
# reach4.ignore_reach()
#-----------------------------------------------
watershed.update_graph()

watershed.update_junctions_and_node_types()
watershed.identify_inflow_sources()

watershed.update_default_us_ds_default_values()

watershed.renumber_all_nodes_and_reaches_in_CCHE1D_computational_order()
watershed.set_node_id_to_compute_id()
#----------------------------------------------

watershed.assign_cross_section_to_all_points()

In [8]:
df_nodes = watershed.create_cche1d_nodes_df()
df_nodes

,ND_ID,ND_FRMNO,ND_TYPE,ND_XC,ND_YC,ND_DSID,ND_USID,ND_US2ID,ND_CSID,ND_RSID,ND_STID
0,1,1,1,400.0,400.0,2,-1,-1,1,-1,1
1,2,2,6,300.0,400.0,3,1,-1,2,-1,1
2,3,3,6,200.0,400.0,4,2,-1,3,-1,1
3,4,4,9,100.0,400.0,4,3,-1,4,-1,1


In [9]:

# Ignoring a specific reach
# reach4.ignore_reach()
# watershed.update_watershed()
# watershed.update_graph()
# watershed.update_junctions_and_node_types()
# watershed.identify_inflow_sources()
# watershed.update_default_us_ds_default_values()
# watershed.renumber_all_nodes_and_reaches_in_CCHE1D_computational_order()
# watershed.set_node_id_to_compute_id()

In [10]:
df_nodes = watershed.create_cche1d_nodes_df()
df_nodes

,ND_ID,ND_FRMNO,ND_TYPE,ND_XC,ND_YC,ND_DSID,ND_USID,ND_US2ID,ND_CSID,ND_RSID,ND_STID
0,1,1,1,400.0,400.0,2,-1,-1,1,-1,1
1,2,2,6,300.0,400.0,3,1,-1,2,-1,1
2,3,3,6,200.0,400.0,4,2,-1,3,-1,1
3,4,4,9,100.0,400.0,4,3,-1,4,-1,1


In [11]:
# watershed.write_cche1d_dat_files(
#     casename="small_junction", output_folder="C:/Users/Luc/Desktop/test_top2cche1d/small_junction/", sep="\t"
# )

# watershed.write_cche1d_dat_files(
#     casename="small_junction_one_removed", output_folder="C:/Users/Luc/Desktop/test_top2cche1d/small_junction_one_removed/", sep="\t"
# )

watershed.write_cche1d_dat_files(
    casename="small_junction_two_removed", output_folder="C:/Users/Luc/Desktop/test_top2cche1d/small_junction_two_removed/", sep="\t"
)